# Standard Model Lagrangian

We will attempt to build all of the Lorentz and gauge invariant terms in the Standard Model Lagrangian. This is done by considering all of the building blocks

$h, \tilde{h}, D_\mu$ have mass dimension 1

$q_L^i, u_R^i, d_R^i, l_L^i, e_R^i$ have mass dimension $\frac{3}{2}$

$G_{\mu \nu}, W_{\mu \nu}, B_{\mu \nu}$ have mass dimension 2

In [2]:
# Building Blocks
# Each block will have an list of all the meaningful parameters associated with the block
# index
# 0 -> mass dimension
# 1 -> transformation under U(1)
# 2 -> transformation under SU(2)
# 3 -> transformation under SU(3)
# 4 -> spin              [m]  U(1)xSU(2)xSU(3) spin
blocks = {'h':          [1  , 1/2 ,  2  ,  1  , 0  ],
         '\\tilde{h}':  [1  ,-1/2 ,  2  ,  1  , 0  ],
         'D_\mu':       [1  ,   0 ,  0  ,  0  , 0  ], # Covariant derivative should not change how an object transforms.
         'q_L^i':       [3/2, 1/6 ,  2  ,  3  , 1/2],
         'u_R^i':       [3/2, 2/3 ,  1  ,  3  , 1/2],
         'd_R^i':       [3/2,-1/3 ,  1  ,  3  , 1/2],
         'l_L^i':       [3/2,-1/2 ,  2  ,  1  , 1/2],
         'e_R^i':       [3/2,  -1 ,  1  ,  1  , 1/2],
         'v_R^i':       [3/2,   0 ,  1  ,  1  , 1/2], # Right handed neutrino might not exist. Or might be dark matter.
         'G_{\mu \\nu}':[2  ,   0 ,  1  ,  8  , 1  ], # SU(3) gauge boson will transform trivially under other gauge transformations
         'W_{\mu \\nu}':[2  ,   0 ,  3  ,  1  , 1  ], # SU(2) gauge boson will transform trivially under other gauge transformations
         'B_{\mu \\nu}':[2  ,   0 ,  1  ,  1  , 1  ]} # U(1) gauge boson will transform trivially under all gauge transformations

In [3]:
# 4 -> spin              [m]  U(1)xSU(2)xSU(3) spin
normal = {'D_\mu':      [2  ,   0 ,  0  ,  0  , 0  ], # Covariant derivative should not change how an object transforms.
         'h':           [2  ,   3 ,  2  ,  1  , 0  ],
         '\\tilde{h}':  [2  ,  -3 ,  2  ,  1  , 0  ],
         'q_L^i':       [3  ,   1 ,  2  ,  3  , 1/2],
         'u_R^i':       [3  ,   4 ,  1  ,  3  , 1/2],
         'd_R^i':       [3  ,  -2 ,  1  ,  3  , 1/2],
         'l_L^i':       [3  ,  -3 ,  2  ,  1  , 1/2],
         'e_R^i':       [3  ,  -6 ,  1  ,  1  , 1/2],
         'v_R^i':       [3  ,   0 ,  1  ,  1  , 1/2], # Right handed neutrino might not exist. Or might be dark matter.
         'G_{\mu \\nu}':[4  ,   0 ,  1  ,  8  , 1  ], # SU(3) gauge boson will transform trivially under other gauge transformations
         'W_{\mu \\nu}':[4  ,   0 ,  3  ,  1  , 1  ], # SU(2) gauge boson will transform trivially under other gauge transformations
         'B_{\mu \\nu}':[4  ,   0 ,  1  ,  1  , 1  ]} # U(1) gauge boson will transform trivially under all gauge transformations

# Conditions for valid terms

- Need mass dimension less than 4
- Need to be Lorentz invariant (even number of Fermions?)
- Needs to be gauge invariant 
    - Even number of SU(3) indices (if transforms non trivially, has indices)
    - Even number of SU(2) indices (if transforms non trivially, has indices)
    - U(1) charge sums to integer
- Each covariant derivative needs to contract with a (two?) Fermion(s) or another covariant derivative 

# How to find all combinations
- Need a way of handling daggars - conjugation
- Need a way of handling indices

could do a terrible brute force and say that since the most fields a term could have would be 4 since every field has mass dimension at leat 1 and we constrict our terms to be less than or equal to 4

In [4]:
#                        [m]  U(1)xSU(2)xSU(3) spin
blocks = {'D_\mu':      [1  ,   0 ,  0  ,  0  , 0  ], # Covariant derivative should not change how an object transforms.
         'h':           [1  , 1/2 ,  2  ,  1  , 0  ],
         '\\tilde{h}':  [1  ,-1/2 ,  2  ,  1  , 0  ],
         'q_L^i':       [3/2, 1/6 ,  2  ,  3  , 1/2],
         'u_R^i':       [3/2, 2/3 ,  1  ,  3  , 1/2],
         'd_R^i':       [3/2,-1/3 ,  1  ,  3  , 1/2],
         'l_L^i':       [3/2,-1/2 ,  2  ,  1  , 1/2],
         'e_R^i':       [3/2,  -1 ,  1  ,  1  , 1/2],
         'v_R^i':       [3/2,   0 ,  1  ,  1  , 1/2], # Right handed neutrino might not exist. Or might be dark matter.
         'G_{\mu \\nu}':[2  ,   0 ,  1  ,  8  , 1  ], # SU(3) gauge boson will transform trivially under other gauge transformations
         'W_{\mu \\nu}':[2  ,   0 ,  3  ,  1  , 1  ], # SU(2) gauge boson will transform trivially under other gauge transformations
         'B_{\mu \\nu}':[2  ,   0 ,  1  ,  1  , 1  ]} # U(1) gauge boson will transform trivially under all gauge transformations

In [5]:
# import copy
def validMass(curr):
    # first check is to see if it has mass dimension less than 4
    massDim = 0
    for t in curr:
        massDim += blocks[t[0]][0]
    
    return massDim <= 4 and massDim%1 == 0

#total spin should be an integer I think?
def LorentzSymmetry(curr):
    totalSpin = 0
    fermionNumber = 0
    lorentzIndex = 0

    for t in curr:
        if blocks[t[0]][4] == 1/2:
            fermionNumber += 1
        totalSpin += blocks[t[0]][4] 
    
    return totalSpin % 1 == 0

def gaugeSymmetry(curr):
    #U(1)
    hyperCharge = 0
    for t in curr:
        hyperCharge += (1-2*t[1])*blocks[t[0]][1]
    
    #SU(2)
    weakTransformation = 0
    for t in curr:
        weakTransformation += blocks[t[0]][2]
    
    #SU(3)
    strongTransformation = 0
    for t in curr:
        strongTransformation += blocks[t[0]][3]
    
    return hyperCharge%1 == 0 and weakTransformation%2 == 0 and strongTransformation%3 == 0

def validTerm(curr):
    return gaugeSymmetry(curr) and LorentzSymmetry(curr) and validMass(curr)
    

In [6]:
validTerm([['h',0],['h',0]])
    #create conjugate to check if valid too
#     conj = new.copy()
#     if

False

In [7]:
# Each term added will be added as a doublet where the second index represents whether (1) or not (0) it has been conjugated - 1/2 if barred
valid = []

# Generate Terms
Now we will loop through all possible combinations of each term and it's conjugate and if it creates a valid term then we will add it to the list

two different methods being considered, brute force - no care for duplicates

or if we consider the two field case, then all the comparrisons we need to do are in the

In [8]:
import copy

def addTerm(curr,valid):
    s = ""
    for f in curr:
        if f[1] == 1:   
            s+= "(" + f[0] + ")^{\\dag} " 
        else: 
            s+= f[0] + " " 
    valid.append(str(s))
    return valid

def termLoop(index,curr,blocks,maxDim,valid):
#     curr = cur.copy()
    if len(curr)>maxDim or index >= len(blocks):
        return 0
    
    for i in range(index,len(blocks)):
        
        curr.append([blocks[i],0])
        if validTerm(curr): 
            valid = addTerm(curr,valid)
        termLoop(index+1,curr,blocks,maxDim,valid)
        curr.pop()
           
        curr.append([blocks[i],1])
        if validTerm(curr):
            valid = addTerm(curr,valid)
        termLoop(index+1,curr,blocks,maxDim,valid)
        curr.pop()
        
    return 0

def generateTerms(blocks):
    cur = []
    valid = []
    blocks = sorted(blocks)
    
    termLoop(0,cur,blocks,4,valid)
#     print(len(valid))
#     mini = list(set(valid)).sort()
    mini = sorted(valid)    
    print(len(mini))
    
    for t in mini: 
        print(t)
        

In [9]:
valid = []
generateTerms(blocks)


68
(B_{\mu \nu})^{\dag} (G_{\mu \nu})^{\dag} 
(B_{\mu \nu})^{\dag} G_{\mu \nu} 
(D_\mu)^{\dag} 
(D_\mu)^{\dag} (D_\mu)^{\dag} 
(D_\mu)^{\dag} (d_R^i)^{\dag} d_R^i 
(D_\mu)^{\dag} (d_R^i)^{\dag} u_R^i 
(D_\mu)^{\dag} (q_L^i)^{\dag} q_L^i 
(D_\mu)^{\dag} (u_R^i)^{\dag} d_R^i 
(D_\mu)^{\dag} (u_R^i)^{\dag} u_R^i 
(D_\mu)^{\dag} D_\mu 
(D_\mu)^{\dag} d_R^i (d_R^i)^{\dag} 
(D_\mu)^{\dag} d_R^i (u_R^i)^{\dag} 
(D_\mu)^{\dag} q_L^i (q_L^i)^{\dag} 
(D_\mu)^{\dag} u_R^i (d_R^i)^{\dag} 
(D_\mu)^{\dag} u_R^i (u_R^i)^{\dag} 
(G_{\mu \nu})^{\dag} (W_{\mu \nu})^{\dag} 
(G_{\mu \nu})^{\dag} W_{\mu \nu} 
(W_{\mu \nu})^{\dag} (G_{\mu \nu})^{\dag} 
(W_{\mu \nu})^{\dag} G_{\mu \nu} 
(d_R^i)^{\dag} (D_\mu)^{\dag} d_R^i 
(d_R^i)^{\dag} (D_\mu)^{\dag} u_R^i 
(d_R^i)^{\dag} D_\mu d_R^i 
(d_R^i)^{\dag} D_\mu u_R^i 
(d_R^i)^{\dag} d_R^i 
(d_R^i)^{\dag} u_R^i 
(q_L^i)^{\dag} (D_\mu)^{\dag} q_L^i 
(q_L^i)^{\dag} D_\mu q_L^i 
(q_L^i)^{\dag} q_L^i 
(u_R^i)^{\dag} (D_\mu)^{\dag} d_R^i 
(u_R^i)^{\dag} (D_\mu)^{\dag}